In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import panel as pn

pn.extension("ipywidgets", "katex", "mathjax")
import sys
from inspect import signature
from random import shuffle, uniform

import ipywidgets as ipw
from matplotlib.animation import FuncAnimation

# from matplotlib.ticker import MultipleLocator
from matplotlib.figure import Figure

print("Packages succesfully loaded")

# Functions from cookbook

In [ ]:
def multiple_selection(question, correct_statements, false_statements):
    # Make an empty list to store the widgets (references), checkboxes, and true/false statements sorted.
    check_boxes = []  # all the boxes to click
    all_statements = []  # all the statements

    # question widget
    question_widget = pn.widgets.StaticText(value=question)

    # for visualization, the maximum length/width of the question
    max_length = max(len(item) for item in correct_statements + false_statements)
    width_statement = max(250, max_length * 7)

    # An empty list for visualization to store the HBoxes that contain the widgets, one statement and the corresponding checkbox
    all_widgets = []

    for statement in correct_statements + false_statements:
        add_statement = pn.widgets.StaticText(value=statement, width=width_statement)
        check_box_widget = pn.widgets.Checkbox(value=False, width=120)
        HBox1 = pn.Row(add_statement, check_box_widget)

        all_statements.append(add_statement)
        check_boxes.append(check_box_widget)
        all_widgets.append(HBox1)

    # randomize the order of statements
    shuffle(all_widgets)

    # add submit button and output, which come on the bottom
    submit_button = pn.widgets.Button(name="Check")
    output_widget = pn.widgets.TextInput(value="", placeholder="", disabled=False)

    # make an additional HBox for aligning the submit button and the output widget
    HBox2 = pn.Row(submit_button, output_widget)
    all_widgets.append(HBox2)

    # align all the HBoxes beneath each other (oldest below if not randomized) and display them.
    quiz_widget = pn.Column(question_widget, *all_widgets)

    # Check the checkbox for each statement and calculate the score.
    def check_answers(event):
        score = 0

        for i in range(len(check_boxes)):
            check_box = check_boxes[i]
            statement = all_statements[i].value

            if statement in correct_statements:
                if check_box.value == True:
                    score += 1
                else:
                    score -= 0

            if statement not in correct_statements:
                if check_box.value == True:
                    score -= 1
                else:
                    score -= 0

        score = np.max([score, 0])
        output_widget.value = (
            f"You have: {round(score/len(correct_statements)*100,0)}% of the points"
        )

    submit_button.on_click(check_answers)
    return quiz_widget  # This is similar to .serve(), it can also be done through: display(quiz_widget)

In [ ]:
def check_code_function(
    fig, horizontal_axis, function_name, correct_function, par_x_axis, f_margin=0.001
):
    # The error margin (f_margin) is set at 0.1% if it is not defined

    # make the graph and show the correct answer
    ax = fig.subplots()
    pane = pn.pane.Matplotlib(fig, dpi=100)

    #  Plot the answer if the students function is found
    try:
        # Load the student-made function from globals
        function = globals()[function_name]

        # Add the arguments to the name of the function, for eval()
        # + Replace the argument on the x-axis so that it can be assessed with list comprehension.
        # https://docs.python.org/3/library/inspect.html
        # https://peps.python.org/pep-0362/
        sig = signature(function)
        sig_function = str(sig).replace(par_x_axis, "par_x_axis")
        function = function_name + str(sig_function)
        title = function_name + str(sig)
        title = title.replace("_", " ")

        sig = signature(correct_function)
        sig_function = str(sig).replace(par_x_axis, "par_x_axis")
        correct_function2 = str(correct_function.__name__) + sig_function

        # calculate the student's answer through eval()
        student_answer = [eval(function) for par_x_axis in horizontal_axis]

        # calculate the correct answer
        correct_answer = []
        for par_x_axis in horizontal_axis:
            correct_answer.append(eval(correct_function2))
        # This code should work, but it does not
        # correct_answer = [eval(correct_function2) for par_x_axis in horizontal_axis]

        # check if the answer is correct and plot it before/below lines
        changes = np.array(student_answer) - np.array(correct_answer)
        inaccuracy = np.abs(1 - np.array(correct_answer) / np.array(student_answer))

        if np.max(changes) == 0:
            y_loc = (np.mean(correct_answer) + np.min(correct_answer)) / 2
            text = ax.text(
                np.mean([horizontal_axis]),
                y_loc,
                "Perfect!",
                fontsize=12,
                color="#1b5a00",
                ha="center",
                va="center",
            )

        if np.max(changes) != 0 and np.max(inaccuracy) < f_margin:
            y_loc = (np.mean(correct_answer) + np.min(correct_answer)) / 2
            text = ax.text(
                np.mean([horizontal_axis]),
                y_loc,
                "Good!",
                fontsize=12,
                color="#1b5a00",
                ha="center",
                va="center",
            )

        # plot the answers
        line = ax.plot(horizontal_axis, student_answer, label="Your answer")
        line = ax.plot(horizontal_axis, correct_answer, label="Correct answer")
        ax.legend()
        ax.set_title(title)

    except:
        text_failed = "Almost there, \n your function can not be plotted, \n please try to fix the bug"
        x_ticks = ax.get_xticks()
        y_ticks = ax.get_yticks()
        text = ax.text(
            np.average(x_ticks),
            np.average(y_ticks),
            text_failed,
            fontsize=16,
            color="r",
            ha="center",
            va="center",
        )

    # update the graph
    display(pane)

## new functions for cookbook

In [ ]:
def check_code_function_RANGE(fig, function_name, correct_function, f_margin=0.001):
    # The error margin (f_margin) is set at 0.1% if it is not defined

    # make the graph and show the correct answer
    ax = fig.subplots()
    pane = pn.pane.Matplotlib(fig, dpi=100)

    #  Plot the answer if the student function is found
    try:
        # Load the student-made function from globals
        function = globals()[function_name]

        # Add the arguments to the name of the function, for eval()
        # https://docs.python.org/3/library/inspect.html
        # https://peps.python.org/pep-0362/
        sig = signature(function)
        student_function = function_name + str(sig)

        student_answer = eval(student_function)

        sig = signature(correct_function)
        correct_function2 = str(correct_function.__name__) + str(sig)
        correct_answer = eval(correct_function2)

        # plot the answers
        line = ax.plot(student_answer, label="Your answer", zorder=1)
        line = ax.plot(correct_answer, label="Correct answer", zorder=1)

        # check if the answer is correct and plot it before/below lines
        changes = np.array(student_answer) - np.array(correct_answer)
        inaccuracy = np.abs(1 - np.array(correct_answer) / np.array(student_answer))

        # Plot comment if the answer is correct
        y_loc = (np.nanmean(correct_answer) + np.nanmin(correct_answer)) / 2
        x_loc = np.mean(ax.get_xticks())
        if np.nanmax(changes) == 0:
            text = ax.text(
                x_loc,
                y_loc,
                "Perfect!",
                fontsize=12,
                color="#1b5a00",
                ha="center",
                va="center",
                zorder=0,
            )

        if np.nanmax(changes) != 0 and np.nanmax(inaccuracy) < f_margin:
            text = ax.text(
                x_loc,
                y_loc,
                "Good!",
                fontsize=12,
                color="#1b5a00",
                ha="center",
                va="center",
                zorder=0,
            )

        # show legend, set title, and update figure
        ax.legend()
        title = student_function
        title = title.replace("_", " ")
        ax.set_title(title)

    except:
        text_failed = "Almost there, \n your function can not be plotted, \n please try to fix the bug"
        x_ticks = ax.get_xticks()
        y_ticks = ax.get_yticks()
        text = ax.text(
            np.average(x_ticks),
            np.average(y_ticks),
            text_failed,
            fontsize=16,
            color="r",
            ha="center",
            va="center",
        )

    # update the graph
    display(pane)

# Commonly used functions

In [ ]:
def wave_length(T, h):
    L = 9.81 * T**2 / (2 * np.pi)
    L_all = [L]

    for i in range(1500):
        L = 9.81 * T**2 / (2 * np.pi) * np.tanh(2 * np.pi * h / L)
        L_all.append(L)

        # stop the iteration when the error is sufficiently small
        if np.abs(L_all[-1] - L_all[-2]) < 0.0005:
            break

    return round(L, 13)


def group_stats(k1, k2, w1, w2):
    Delta_k = np.abs(k2 - k1)
    Delta_w = np.abs(w2 - w1)
    L = 2 * np.pi / Delta_k
    T = 2 * np.pi / Delta_w
    cg = Delta_w / Delta_k
    return L, T, cg

# Part 1B

In [ ]:
def W3_fig_5_13():
    a1 = ipw.FloatText(value=1, min=0, max=20, step=0.01, description="a [m]")
    a2 = ipw.FloatText(value=0.25, min=0, max=20, step=0.01, description="a [m]")

    # Setup widget layout (User Interface)
    vbox1 = ipw.VBox([ipw.Label("Wave 1", layout=ipw.Layout(align_self="center")), a1])
    vbox2 = ipw.VBox([ipw.Label("Wave 2", layout=ipw.Layout(align_self="center")), a2])
    ui = ipw.HBox([vbox1, vbox2])

    def update_graph(a1, a2):
        S_min = -0.25 * np.pi
        S_max = 2.25 * np.pi

        S = np.linspace(S_min, S_max, 60)

        eta1 = a1 * np.cos(S) / a1
        eta2 = a2 * np.cos(S * 2) / a1
        eta = eta1 + eta2

        fig, axs = plt.subplots(
            nrows=1, ncols=1, figsize=(5, 4), sharex=False, sharey=False
        )
        axs.plot(S, eta1, label="$\eta_1$", color="gray", linestyle="dashed")
        axs.plot(S, eta2, label="$\eta_2$", color="gray", linestyle="dashdot")
        axs.plot(S, eta, label="$\eta_{1+2}$", color="k")
        axs.legend(loc="best")
        axs.set_ylabel("eta/eta_1 [-]")
        axs.set_xlabel("phase [rad]")
        axs.set_xlim(S_min, S_max)

        x_tick_min = S_min // (0.5 * np.pi) * 0.5 * np.pi + 0.5 * np.pi
        x_ticks = np.arange(x_tick_min, S_max, 0.5 * np.pi)
        x_ticks_labels = [f"{angle/np.pi:.1f}π" for angle in x_ticks]
        axs.set_xticks(x_ticks, x_ticks_labels)

    graph = ipw.interactive_output(update_graph, {"a1": a1, "a2": a2})
    display(ui, graph)


# W3_fig_5_13()

In [ ]:
def W3_fig_5_14():
    a1 = ipw.FloatText(value=1, min=0, max=20, step=0.01, description="a1 [m]")
    ui = a1

    def update_graph(a1):
        S_min = -0.25 * np.pi
        S_max = 2.25 * np.pi

        S = np.linspace(S_min, S_max, 60)

        # left graph, sinusoidal to the power 1 and 3
        eta1 = a1 * np.cos(S) / a1
        eta1_3 = eta1**3

        # right graph, 2 harmonic components (2nd order stokes) to the first and third order
        a2 = 0.25 * a1
        eta = (a1 * np.cos(S) + a2 * np.cos(S * 2)) / a1
        eta_3 = eta**3

        # set structure for graph
        fig, axs = plt.subplots(
            nrows=1, ncols=2, figsize=(10, 4), sharex=False, sharey=True
        )
        fig.subplots_adjust(hspace=0)
        fig.subplots_adjust(wspace=0.06)

        # plot lines
        axs[0].plot(S, eta1, label="$\eta_1$", color="gray", linestyle="-")
        axs[0].plot(S, eta1_3, label="$\eta^3_1$", color="k", linestyle="-")
        axs[0].plot([S_min, S_max], [0, 0], color="k", linewidth=1, linestyle="dashed")

        axs[1].plot(S, eta, label="$\eta$", color="gray", linestyle="-")
        axs[1].plot(S, eta_3, label="$\eta^3$", color="k", linestyle="-")
        axs[1].plot([S_min, S_max], [0, 0], color="k", linewidth=1, linestyle="dashed")

        # set titles
        axs[0].set_title("Harmonic component \n (sinusoidal)")
        axs[1].set_title("Second-order Stokes wave \n (skewed)")

        # set legend and labels
        axs[0].legend(loc="best")
        axs[0].set_ylabel("$\eta/\eta_1$ [-]")
        axs[0].set_xlabel("phase [rad]")
        axs[0].set_xlim(S_min, S_max)

        axs[1].legend(loc="best")
        axs[1].set_xlabel("phase [rad]")
        axs[1].set_xlim(S_min, S_max)

        x_tick_min = S_min // (0.5 * np.pi) * 0.5 * np.pi + 0.5 * np.pi
        x_ticks = np.arange(x_tick_min, S_max, 0.5 * np.pi)
        x_ticks_labels = [f"{angle/np.pi:.1f}π" for angle in x_ticks]
        axs[0].set_xticks(x_ticks, x_ticks_labels)
        axs[1].set_xticks(x_ticks, x_ticks_labels)

    graph = ipw.interactive_output(update_graph, {"a1": a1})
    display(ui, graph)


# W3_fig_5_14()

In [ ]:
def W3_fig_5_15():
    a1 = ipw.FloatText(value=1, min=0, max=20, step=0.01, description="a1 [m]")
    ui = a1

    def update_graph(a1):
        T1, depth, n_waves = 4, 9999, 1
        T2 = 0.5 * T1
        a2 = 0.25 * a1

        L1 = wave_length(T1, depth)
        L2 = 0.5 * L1

        L_group, T_group, c_g = group_stats(
            k1=2 * np.pi / L1,
            k2=2 * np.pi / L2,
            w1=2 * np.pi / T1,
            w2=2 * np.pi / T2,
        )

        t = np.arange(0, n_waves * T1 + T1 / 60, T1 / 60)
        x = np.arange(0, n_waves * L1 + L1 / 60, L1 / 60)

        xp, tp = 0, 0
        eta1_T = a1 / a1 * np.cos(2 * np.pi / T1 * t - 2 * np.pi / L1 * xp)
        eta2_T = (
            a2 / a1 * np.cos(2 * np.pi / T2 * t - 2 * np.pi / L2 * xp - 0.5 * np.pi)
        )
        eta_T = eta1_T + eta2_T

        eta1_x = a1 / a1 * np.cos(2 * np.pi / T1 * tp - 2 * np.pi / L1 * x)
        eta2_x = (
            a2 / a1 * np.cos(2 * np.pi / T2 * tp - 2 * np.pi / L2 * x - 0.5 * np.pi)
        )
        eta_x = eta1_x + eta2_x

        # set structure for graph
        fig, axs = plt.subplots(
            nrows=1, ncols=2, figsize=(10, 4), sharex=False, sharey=True
        )
        fig.subplots_adjust(hspace=0)
        fig.subplots_adjust(wspace=0.06)

        # plot lines
        axs[0].plot(t, eta1_T, label="$\eta_1$", color="gray", linestyle="dashed")
        axs[0].plot(t, eta2_T, label="$\eta_2$", color="gray", linestyle="dashdot")
        axs[0].plot(t, eta_T, label="$\eta$", color="k", linestyle="-")

        axs[1].plot(x, eta1_x, label="$\eta_1$", color="gray", linestyle="dashed")
        axs[1].plot(x, eta2_x, label="$\eta_2$", color="gray", linestyle="dashdot")
        axs[1].plot(x, eta_x, label="$\eta$", color="k", linestyle="-")

        # set x-lim
        axs[0].set_xlim(0, T1 * n_waves)
        axs[1].set_xlim(0, L1 * n_waves)

        # set x-ticks, 4 every wave period/length
        x_ticks = np.arange(0, n_waves + 0.25, 0.25 * n_waves)
        x_values_1 = x_ticks * n_waves * T1
        x_values_2 = x_ticks * n_waves * L1

        axs[0].set_xticks(x_values_1, x_ticks)
        axs[1].set_xticks(x_values_2, x_ticks)

        # set labels
        axs[0].set_xlabel("t/T1 [-]")
        axs[1].set_xlabel("x/L1 [-]")

        axs[0].set_ylabel("$\eta/\eta_1$ [-]")

        # set legend
        axs[0].legend(loc="best")
        axs[1].legend(loc="best")

        # set title
        axs[0].set_title("Time based")
        axs[1].set_title("Space based")

    graph = ipw.interactive_output(update_graph, {"a1": a1})
    display(ui, graph)


# W3_fig_5_15()

In [ ]:
def W3_fig_5_16():
    a1 = ipw.FloatText(value=1, min=0, max=20, step=0.01, description="a1 [m]")
    ui = a1

    def update_graph(a1):
        S_min = -0.25 * np.pi
        S_max = 2.25 * np.pi

        # wave phases
        S = np.linspace(S_min, S_max, 60)
        S2 = S * 2 - 0.5 * np.pi  # twice as fast and 90 degrees phase forwarded

        # left graph, sinusoidal to the power 1 and 3
        eta1 = a1 * np.cos(S) / a1
        eta1_3 = eta1**3

        # right graph, 2 harmonic components (2nd order stokes) to the first and third order
        a2 = 0.25 * a1
        eta_1 = a1 * np.cos(S) / a1
        eta_2 = a2 * np.cos(S2) / a1
        eta = eta_1 + eta_2

        # set structure for graph
        fig, axs = plt.subplots(
            nrows=1, ncols=1, figsize=(5, 4), sharex=False, sharey=True
        )
        fig.subplots_adjust(hspace=0)
        fig.subplots_adjust(wspace=0.06)

        # plot lines
        axs.plot(S, eta_1, label="$\eta_1$", color="gray", linestyle="dashed")
        axs.plot(S, eta_2, label="$\eta_2$", color="gray", linestyle="dashdot")
        axs.plot(S, eta, label="$\eta$", color="k", linestyle="-")

        # set legend and labels
        axs.legend(loc="best")
        axs.set_ylabel("$\eta/\eta_1$ [-]")
        axs.set_xlabel("phase [rad]")
        axs.set_xlim(S_min, S_max)

        x_tick_min = S_min // (0.5 * np.pi) * 0.5 * np.pi + 0.5 * np.pi
        x_ticks = np.arange(x_tick_min, S_max, 0.5 * np.pi)
        x_ticks_labels = [f"{angle/np.pi:.1f}π" for angle in x_ticks]
        axs.set_xticks(x_ticks, x_ticks_labels)

    graph = ipw.interactive_output(update_graph, {"a1": a1})
    display(ui, graph)


# W3_fig_5_16()

In [ ]:
def W3_fig_5_17():
    a1 = ipw.FloatText(value=1, min=0, max=20, step=0.01, description="a1 [m]")
    ui = a1

    def update_graph(a1):
        S_min = -0.25 * np.pi
        S_max = 2.25 * np.pi

        S = np.linspace(S_min, S_max, 60)
        S2 = S * 2 + 0.5 * np.pi  # twice as fast and 90 degrees phase forwarded

        # left graph, 2 harmonic components (2nd order stokes) to the first and third order
        a2 = 0.25 * a1
        eta = (a1 * np.cos(S) + a2 * np.cos(S * 2)) / a1
        eta_3 = eta**3

        # right graph, assymetric
        eta_As = (a1 * np.cos(S) + a2 * np.cos(S2)) / a1
        eta_As_3 = eta_As**3

        # set structure for graph
        fig, axs = plt.subplots(
            nrows=1, ncols=2, figsize=(10, 4), sharex=False, sharey=True
        )
        fig.subplots_adjust(hspace=0)
        fig.subplots_adjust(wspace=0.06)

        # plot lines
        axs[0].plot(S, eta, label="$\eta$", color="gray", linestyle="-")
        axs[0].plot(S, eta_3, label="$\eta^3$", color="k", linestyle="-")
        axs[0].plot([S_min, S_max], [0, 0], color="k", linewidth=1, linestyle="dashed")

        axs[1].plot(S, eta_As, label="$\eta$", color="gray", linestyle="-")
        axs[1].plot(S, eta_As_3, label="$\eta^3$", color="k", linestyle="-")
        axs[1].plot([S_min, S_max], [0, 0], color="k", linewidth=1, linestyle="dashed")

        # set titles
        axs[0].set_title("Skewed waves \n (Second-order Stokes wave)")
        axs[0].plot([S_min, S_max], [0, 0], color="k", linewidth=1, linestyle="dashed")
        # set legend and labels
        axs[0].legend(loc="best")
        axs[0].set_ylabel("$\eta/\eta_1$ [-]")
        axs[0].set_xlabel("phase [rad]")
        axs[0].set_xlim(S_min, S_max)

        axs[1].set_title("Assymetric waves")
        axs[1].legend(loc="best")
        axs[1].set_xlabel("phase [rad]")
        axs[1].set_xlim(S_min, S_max)

        x_tick_min = S_min // (0.5 * np.pi) * 0.5 * np.pi + 0.5 * np.pi
        x_ticks = np.arange(x_tick_min, S_max, 0.5 * np.pi)
        x_ticks_labels = [f"{angle/np.pi:.1f}π" for angle in x_ticks]
        axs[0].set_xticks(x_ticks, x_ticks_labels)
        axs[1].set_xticks(x_ticks, x_ticks_labels)

    graph = ipw.interactive_output(update_graph, {"a1": a1})
    display(ui, graph)


# W3_fig_5_17()

# Part 2

### Normal incident waves

In [ ]:
def W3_interactiveplot_normal_indicent_waves(H0, T, d0, slope_in):

    slope = 1.0 / slope_in  # bed slope [-]
    d0  # offshore water depth [m]
    x_max = round((d0 + 2) / slope)
    x = np.arange(0, x_max + 1, 1)  # cross-shore coordinate [m]
    zbed = -(d0 - slope * x)  # bed elevation [m]
    h = -zbed  # still water depth [m]
    h[h < 0] = 0  # no negative depths

    # w is zero when h is 0, causing a divide by zero.
    # shorten the lists if a water depth of 0 is reached.
    x0_id = np.argwhere(h == 0)[0][0]  # first location where water depth = 0
    h = h[0:x0_id]
    x_water = x[0:x0_id]

    # given:
    gamma = 0.8  # wave breaking ratio

    # The wave characteristics at every location in the cross-section
    L = np.array([wave_length(T, h) for h in h])  # The wave length
    c = L / T  # The wave celerity
    k = 2 * np.pi / L  # The wave number
    n = 0.5 + (k * h / np.sinh(2 * k * h))
    cg = n * c  # The wave group celerity
    Ksh = np.sqrt(cg[0] / cg)  # The shoaling parameter
    H = H0 * Ksh  # The wave height due to shoaling (only)
    H_shoal = H.copy()
    Hbreaking = gamma * h  # The wave-breaking height
    H[H > Hbreaking] = Hbreaking[H > Hbreaking]  # The wave height

    fig, axs = plt.subplots(nrows=4, ncols=2, figsize=(9, 6), sharex=True, sharey=False)
    fig.subplots_adjust(hspace=0)
    fig.subplots_adjust(wspace=0.15)

    axs[0, 0].plot(
        x, zbed, label="Bed level [m] (1:" + str(round(slope_in, 2)) + ")", color="k"
    )
    axs[0, 0].plot(
        [0, x[x0_id]], [0, 0], color="gray", label="Still water elevation [m]"
    )
    axs[1, 0].plot(x_water, L, label="Wavelength (L) [m]", color="k")
    axs[2, 0].plot(x_water, k, label="Wave number (k) [rad/m]", color="k")
    axs[3, 0].plot(x_water, c, label="Wave celerity (c) [m]", color="k")
    axs[0, 1].plot(x_water, cg, label="Wavegroup celerity (cg) [m/s]", color="k")
    axs[1, 1].plot(x_water, Ksh, label="shoaling factor (ksh) [-]", color="k")
    axs[2, 1].plot(x_water, H_shoal, label="only shoaling wave height [m]", color="k")
    axs[3, 1].plot(x_water, H, label="breaking wave height [m]", color="k")

    for ax in axs:
        ax[0].legend(loc="lower left")
        ax[0].xaxis.set_visible(False)
        ax[1].legend(loc="best")
        ax[1].xaxis.set_visible(False)
    axs[3, 0].xaxis.set_visible(True)
    axs[3, 1].xaxis.set_visible(True)

    # print(f'max group celerity: {np.max(cg)}')
    # print(f'wave height: {np.max(H)}')
    # xid = np.where(H < H_shoal)[0][0]
    # print(f'widht surfzone = from {x_water[xid]} to {x_water[-1]} is {- x_water[xid] + x_water[-1]}')


def W3_plot_normal_indicent_waves():
    # Create interactive widgets, which require IPY Widgets, widgets from panel do not work
    # H0 = pn.widgets.FloatInput(value=1.5, start=0, end=500, step=0.1, name="Offshore wave height (H0) [m]")
    # T = pn.widgets.FloatInput(value=5, start=0.05, end=500, step=0.01, name ="Wave period (T) [s]")
    # slope = pn.widgets.FloatInput(value=30, start=0.1, end=50, step=0.1, name ="slope 1:...")
    # d0 = pn.widgets.FloatInput(value=50, start=0.1, end =500, step=0.1, name="offshore depth (h0) [m]")

    H0 = ipw.FloatText(
        value=1.5, min=0, max=500, step=0.1, description="H0 [m]", width=75
    )
    T = ipw.FloatText(value=6, min=0.05, max=500, step=0.01, description="T [s]")
    slope = ipw.FloatText(
        value=30, min=0.1, max=50, step=0.1, description="slope 1:..."
    )
    d0 = ipw.FloatText(value=50, min=0.1, max=500, step=0.1, description="depth [m]")

    Vbox1 = ipw.VBox([H0, T])
    Vbox2 = ipw.VBox([d0, slope])

    widgets = ipw.HBox([Vbox1, Vbox2])
    graph = ipw.interactive_output(
        W3_interactiveplot_normal_indicent_waves,
        {"H0": H0, "T": T, "d0": d0, "slope_in": slope},
    )

    display(widgets, graph)


# W3_plot_normal_indicent_waves()

In [ ]:
def W3_Q4():
    W3_plot_normal_indicent_waves()

    Q1_text = "Which method is used to calculate the wavelength?"
    Q1_cor = ["Iterative approach"]
    Q1_false = ["Linear interpolating tables", "An explicit formula"]

    Q2_text = "Which increases the width of the shoaling zone?"
    Q2_cor = ["Increasing wave period (T)", "Decreasing bed slope"]
    Q2_false = ["Increasing offshore wave height (H0)", "Increasing deep water depth"]

    Q3_text = "Which increases the wave breaking height?"
    Q3_cor = [" A larger offshore wave height (H0)", " A larger wave period (T)"]
    Q3_false = ["A steeper bed slope", "Increasing the deep water depth"]

    # Q4_text = 'Which influences the set-down?'
    # Q4_cor = ['The wave period (T)']
    # Q4_false = ['The deepwater wave height (H0)', 'The bed slope', 'The deep water depth']

    Q4_text = "Which of the following changes in parameters will increase the width of the surf zone"
    Q4_cor = [
        "Increasing deepwater wave height (H0)",
        "Increasing the wave period (T)",
        "Decreasing the bed slope",
    ]
    Q4_false = ["Decreasing the deep water depth"]

    # Visualize it through a list (disabled, now uses eval())
    # Q_texts = [Q1_text, Q2_text, Q3_text, Q4_text, Q5_text]
    # Q_cor_all = [Q1_cor, Q2_cor, Q3_cor, Q4_cor, Q5_cor]
    # Q_false_all = [Q1_false, Q2_false, Q3_false, Q4_false, Q5_false]

    # for Q_text, Q_cor, Q_false in zip(Q_texts, Q_cor_all, Q_false_all):
    #    display(multiple_selection(Q_text, Q_cor, Q_false))

    n_questions = 4
    for i in np.arange(1, n_questions + 1, 1):
        Q = eval("Q" + str(i) + "_text")
        C = eval("Q" + str(i) + "_cor")
        F = eval("Q" + str(i) + "_false")
        display(multiple_selection(Q, C, F))


# W3_Q4()

## Oblique waves

### Wave height

In [ ]:
def W3_plot_oblique_waves():
    # define the name of the function that the students will make
    function_name = "W3_oblique_waves"

    # define the correct function
    def correct_function(x_range, H0, T, d0, slope, angle):

        # The environmental conditions
        x = x_range  # the horizontal axis
        zbed = -(d0 - slope * x)  # bed elevation [m]
        h = -zbed  # still water depth [m]
        h[h < 0] = 0  # no negative depths

        # given conditions
        gamma = 0.8  # wave breaking ratio

        # The wave characteristics at every location in the cross-section
        L = np.array([wave_length(T, h) for h in h])  # The wavelength
        c = L / T  # The wave celerity
        k = 2 * np.pi / L  # The wave number
        n = 0.5 + (k * h / np.sinh(2 * k * h))
        cg = n * c  # The wave group celerity
        Ksh = np.sqrt(cg[0] / cg)  # The shoaling parameter

        """Completed the code here"""
        snell_constant = np.sin(np.deg2rad(angle)) / c[0]  # apply snell's law
        theta_radians = np.arcsin(snell_constant * c)
        Kr = (np.cos(theta_radians[0]) / np.cos(theta_radians)) ** 0.5

        H = H0 * Ksh * Kr  # The wave height due to shoaling and refraction
        Hbreaking = gamma * h  # The wave-breaking height
        H[H > Hbreaking] = Hbreaking[H > Hbreaking]  # Adjusting the wave height

        return H

    # set the acceptable computational error (ratio)
    f_margin = 0.001  # 0.001 = 0.01%

    # set the size of the figure
    fig = Figure((5, 2.5))

    # call the function that builds the backend.
    check_code_function_RANGE(fig, function_name, correct_function, f_margin)

### Radiation stress

In [ ]:
def W3_plot_radiation_stres_Sxx():
    # define the name of the function that the students will make
    function_name = "W3_radiation_stres_Sxx"

    # define the correct function
    def correct_function(x_range, H0, T, d0, slope, angle):

        # The environmental conditions
        x = x_range  # the horizontal axis
        zbed = -(d0 - slope * x)  # bed elevation [m]
        h = -zbed  # still water depth [m]
        h[h < 0] = 0  # no negative depths

        # given conditions
        gamma = 0.8  # wave breaking ratio

        # The wave characteristics at every location in the cross-section
        L = np.array([wave_length(T, h) for h in h])  # The wavelength
        c = L / T  # The wave celerity
        k = 2 * np.pi / L  # The wave number
        n = 0.5 + (k * h / np.sinh(2 * k * h))
        cg = n * c  # The wave group celerity
        Ksh = np.sqrt(cg[0] / cg)  # The shoaling parameter

        """Completed the code here"""
        snell_constant = np.sin(np.deg2rad(angle)) / c[0]  # apply snell's law
        theta_radians = np.arcsin(snell_constant * c)
        Kr = (np.cos(theta_radians[0]) / np.cos(theta_radians)) ** 0.5

        H = H0 * Ksh * Kr  # The wave height due to shoaling and refraction
        Hbreaking = gamma * h  # The wave-breaking height
        H[H > Hbreaking] = Hbreaking[H > Hbreaking]  # Adjusting the wave height

        g = 9.81
        E = 1 / 8 * rho * g * H**2  # The wave energy
        Sxx = (n - 0.5 + n * np.cos(theta_radians) ** 2) * E  # Radiant stresses

        return Sxx

    # set the acceptable computational error (ratio)
    f_margin = 0.001  # 0.001 = 0.01%

    # set the size of the figure
    fig = Figure((5, 2.5))

    # call the function that builds the backend.
    check_code_function_RANGE(fig, function_name, correct_function, f_margin)

In [ ]:
# An example of the student's answer
H0, T, d0, slope, angle, rho = 1.5, 6, 20, 1 / 30, 10, 1025
x_range = np.arange(0, 900, 1)


def W3_radiation_stres_Sxx(x_range, H0, T, d0, slope, angle, rho):

    # The environmental conditions
    x = x_range  # the horizontal axis
    zbed = -(d0 - slope * x)  # bed elevation [m]
    h = -zbed  # still water depth [m]
    h[h < 0] = 0  # no negative depths

    # given conditions
    gamma = 0.8  # wave breaking ratio

    # The wave characteristics at every location in the cross-section
    L = np.array([wave_length(T, h) for h in h])  # The wavelength
    c = L / T  # The wave celerity
    k = 2 * np.pi / L  # The wave number
    n = 0.5 + (k * h / np.sinh(2 * k * h))
    cg = n * c  # The wave group celerity
    Ksh = np.sqrt(cg[0] / cg)  # The shoaling parameter

    """Completed the code here"""
    snell_constant = np.sin(np.deg2rad(angle)) / c[0]  # apply snell's law
    theta_radians = np.arcsin(snell_constant * c)
    Kr = (np.cos(theta_radians[0]) / np.cos(theta_radians)) ** 0.5

    H = H0 * Ksh * Kr  # The wave height due to shoaling and refraction
    Hbreaking = gamma * h  # The wave-breaking height
    H[H > Hbreaking] = Hbreaking[H > Hbreaking]  # Adjusting the wave height

    g = 9.81
    E = 1 / 8 * rho * g * H**2  # The wave energy
    Sxx = (2 * n - 0.5) * E  # Radiant stresses for oblique waves
    Sxx = (n - 0.5 + n * np.cos(theta_radians) ** 2) * E  # Radiant stresses

    return Sxx


# W3_plot_radiation_stres_Sxx()

### Setup

In [ ]:
H0, T, d0, slope, angle, rho = 1.5, 6, 20, 1 / 30, 10, 1025
x_range = np.arange(500, 900, 1)


def W3_plot_wave_setup():
    # define the name of the function that the students will make
    function_name = "W3_wave_setup"

    # define the correct function
    def correct_function(x_range, H0, T, d0, slope, angle):

        # The environmental conditions
        x = x_range  # the horizontal axis
        zbed = -(d0 - slope * x)  # bed elevation [m]
        h = -zbed  # still water depth [m]
        h[h < 0] = 0  # no negative depths

        # given conditions
        gamma = 0.8  # wave breaking ratio

        # The wave characteristics at every location in the cross-section
        L = np.array([wave_length(T, h) for h in h])  # The wavelength
        c = L / T  # The wave celerity
        k = 2 * np.pi / L  # The wave number
        n = 0.5 + (k * h / np.sinh(2 * k * h))
        cg = n * c  # The wave group celerity
        Ksh = np.sqrt(cg[0] / cg)  # The shoaling parameter

        snell_constant = np.sin(np.deg2rad(angle)) / c[0]  # apply snell's law
        theta_radians = np.arcsin(snell_constant * c)
        Kr = (np.cos(theta_radians[0]) / np.cos(theta_radians)) ** 0.5

        H = H0 * Ksh * Kr  # The wave height due to shoaling and refraction
        Hbreaking = gamma * h  # The wave-breaking height
        H[H > Hbreaking] = Hbreaking[H > Hbreaking]  # Adjusting the wave height
        g = 9.81
        E = 1 / 8 * rho * g * H**2  # The wave energy

        # Sxx = (2*n-0.5)*E                             # Radiant stresses for oblique waves
        Sxx = (n - 0.5 + n * np.cos(theta_radians) ** 2) * E  # Radiant stresses

        setup = np.zeros(Sxx.shape)  # here we create a vector for the mean water level
        for i in range(len(setup) - 1):  # key here is that setup[0] = 0
            setup[i + 1] = setup[i] - (Sxx[i + 1] - Sxx[i]) / (1000 * g * h[i])

        return setup

    # set the acceptable computational error (ratio)
    f_margin = 0.001  # 0.001 = 0.01%

    # set the size of the figure
    fig = Figure((5, 2.5))

    # call the function that builds the backend.
    check_code_function_RANGE(fig, function_name, correct_function, f_margin)

Possible student answer

### Nearshore force (Fx)

In [ ]:
H0, T, d0, slope, angle, rho = 1.5, 6, 20, 1 / 30, 10, 1025
x_range = np.arange(0, 900, 1)


def W3_plot_Fx():
    # define the name of the function that the students will make
    function_name = "W3_Fx"

    # define the correct function
    def correct_function(x_range, H0, T, d0, slope, angle):

        # The environmental conditions
        x = x_range  # the horizontal axis
        zbed = -(d0 - slope * x)  # bed elevation [m]
        h = -zbed  # still water depth [m]
        h[h < 0] = 0  # no negative depths

        # given conditions
        gamma = 0.8  # wave breaking ratio

        # The wave characteristics at every location in the cross-section
        L = np.array([wave_length(T, h) for h in h])  # The wavelength
        c = L / T  # The wave celerity
        k = 2 * np.pi / L  # The wave number
        n = 0.5 + (k * h / np.sinh(2 * k * h))
        cg = n * c  # The wave group celerity
        Ksh = np.sqrt(cg[0] / cg)  # The shoaling parameter

        snell_constant = np.sin(np.deg2rad(angle)) / c[0]  # apply snell's law
        theta_radians = np.arcsin(snell_constant * c)
        Kr = (np.cos(theta_radians[0]) / np.cos(theta_radians)) ** 0.5

        H = H0 * Ksh * Kr  # The wave height due to shoaling and refraction
        Hbreaking = gamma * h  # The wave-breaking height
        H[H > Hbreaking] = Hbreaking[H > Hbreaking]  # Adjusting the wave height
        g = 9.81
        E = 1 / 8 * rho * g * H**2  # The wave energy

        # Sxx = (2*n-0.5)*E                             # Radiant stresses for oblique waves
        Sxx = (n - 0.5 + n * np.cos(theta_radians) ** 2) * E  # Radiant stresses

        setup = np.zeros(Sxx.shape)  # here we create a vector for the mean water level
        for i in range(len(setup) - 1):  # key here is that setup[0] = 0
            setup[i + 1] = setup[i] - (Sxx[i + 1] - Sxx[i]) / (1000 * g * h[i])

        Fx = np.zeros(setup.shape)
        for i in range(len(Fx) - 1):
            Fx[i] = -(Sxx[i + 1] - Sxx[i]) / (x[i + 1] - x[i])

        return Fx

    # set the acceptable computational error (ratio)
    f_margin = 0.001  # 0.001 = 0.01%

    # set the size of the figure
    fig = Figure((5, 2.5))

    # call the function that builds the backend.
    check_code_function_RANGE(fig, function_name, correct_function, f_margin)

### Syx

In [ ]:
H0, T, d0, slope, angle, rho = 1.5, 6, 20, 1 / 30, 10, 1025
x_range = np.arange(0, 900, 1)


def W3_plot_Syx():
    # define the name of the function that the students will make
    function_name = "W3_Syx"

    # define the correct function
    def correct_function(x_range, H0, T, d0, slope, angle):

        # The environmental conditions
        x = x_range  # the horizontal axis
        zbed = -(d0 - slope * x)  # bed elevation [m]
        h = -zbed  # still water depth [m]
        h[h < 0] = 0  # no negative depths

        # given conditions
        gamma = 0.8  # wave breaking ratio

        # The wave characteristics at every location in the cross-section
        L = np.array([wave_length(T, h) for h in h])  # The wavelength
        c = L / T  # The wave celerity
        k = 2 * np.pi / L  # The wave number
        n = 0.5 + (k * h / np.sinh(2 * k * h))
        cg = n * c  # The wave group celerity
        Ksh = np.sqrt(cg[0] / cg)  # The shoaling parameter

        snell_constant = np.sin(np.deg2rad(angle)) / c[0]  # apply snell's law
        theta_radians = np.arcsin(snell_constant * c)
        Kr = (np.cos(theta_radians[0]) / np.cos(theta_radians)) ** 0.5

        H = H0 * Ksh * Kr  # The wave height due to shoaling and refraction
        Hbreaking = gamma * h  # The wave-breaking height
        H[H > Hbreaking] = Hbreaking[H > Hbreaking]  # Adjusting the wave height
        g = 9.81
        E = 1 / 8 * rho * g * H**2  # The wave energy

        Syx = n * np.cos(theta_radians) * np.sin(theta_radians) * E

        return Syx

    # set the acceptable computational error (ratio)
    f_margin = 0.001  # 0.001 = 0.01%

    # set the size of the figure
    fig = Figure((5, 2.5))

    # call the function that builds the backend.
    check_code_function_RANGE(fig, function_name, correct_function, f_margin)

### Fy 

In [ ]:
H0, T, d0, slope, angle, rho = 1.5, 6, 20, 1 / 30, 10, 1025
x_range = np.arange(0, 900, 1)


def W3_plot_Fy():
    # define the name of the function that the students will make
    function_name = "W3_Fy"

    # define the correct function
    def correct_function(x_range, H0, T, d0, slope, angle):

        # The environmental conditions
        x = x_range  # the horizontal axis
        zbed = -(d0 - slope * x)  # bed elevation [m]
        h = -zbed  # still water depth [m]
        h[h < 0] = 0  # no negative depths

        # given conditions
        gamma = 0.8  # wave breaking ratio

        # The wave characteristics at every location in the cross-section
        L = np.array([wave_length(T, h) for h in h])  # The wavelength
        c = L / T  # The wave celerity
        k = 2 * np.pi / L  # The wave number
        n = 0.5 + (k * h / np.sinh(2 * k * h))
        cg = n * c  # The wave group celerity
        Ksh = np.sqrt(cg[0] / cg)  # The shoaling parameter

        snell_constant = np.sin(np.deg2rad(angle)) / c[0]  # apply snell's law
        theta_radians = np.arcsin(snell_constant * c)
        Kr = (np.cos(theta_radians[0]) / np.cos(theta_radians)) ** 0.5

        H = H0 * Ksh * Kr  # The wave height due to shoaling and refraction
        Hbreaking = gamma * h  # The wave-breaking height
        H[H > Hbreaking] = Hbreaking[H > Hbreaking]  # Adjusting the wave height
        g = 9.81
        E = 1 / 8 * rho * g * H**2  # The wave energy

        Syx = n * np.cos(theta_radians) * np.sin(theta_radians) * E

        Fy = np.zeros(Syx.shape)
        for i in range(len(Fy) - 1):
            Fy[i] = -(Syx[i + 1] - Syx[i]) / (x[i + 1] - x[i])

        return Fy

    # set the acceptable computational error (ratio)
    f_margin = 0.001  # 0.001 = 0.01%

    # set the size of the figure
    fig = Figure((5, 2.5))

    # call the function that builds the backend.
    check_code_function_RANGE(fig, function_name, correct_function, f_margin)

### Longshore current V(x)

In [ ]:
H0, T, d0, slope, angle, rho, r = 1.5, 6, 20, 1 / 30, 10, 1025, 0.06
x_range = np.arange(0, 900, 1)


def W3_plot_V():
    # define the name of the function that the students will make
    function_name = "W3_V"

    # define the correct function
    def correct_function(x_range, H0, T, d0, slope, angle, r):

        # The environmental conditions
        x = x_range  # the horizontal axis
        zbed = -(d0 - slope * x)  # bed elevation [m]
        h = -zbed  # still water depth [m]
        h[h < 0] = 0  # no negative depths

        # given conditions
        gamma = 0.8  # wave breaking ratio

        # The wave characteristics at every location in the cross-section
        L = np.array([wave_length(T, h) for h in h])  # The wavelength
        c = L / T  # The wave celerity
        k = 2 * np.pi / L  # The wave number
        n = 0.5 + (k * h / np.sinh(2 * k * h))
        cg = n * c  # The wave group celerity
        Ksh = np.sqrt(cg[0] / cg)  # The shoaling parameter

        snell_constant = np.sin(np.deg2rad(angle)) / c[0]  # apply snell's law
        theta_radians = np.arcsin(snell_constant * c)
        Kr = (np.cos(theta_radians[0]) / np.cos(theta_radians)) ** 0.5

        H = H0 * Ksh * Kr  # The wave height due to shoaling and refraction
        Hbreaking = gamma * h  # The wave-breaking height
        H[H > Hbreaking] = Hbreaking[H > Hbreaking]  # Adjusting the wave height
        g = 9.81
        E = 1 / 8 * rho * g * H**2  # The wave energy

        Syx = n * np.cos(theta_radians) * np.sin(theta_radians) * E

        dh_dx = np.zeros(len(h))  # dh/dx
        for i in range(len(dh_dx) - 1):
            dh_dx[i] = (h[i + 1] - h[i]) / (x[i + 1] - x[i])

        # page 198
        # r = 0.06                                  # bottom roughness [m]
        C = 18 * np.log(12) * h / r  # Chezy coeffcient
        c_f = g / C**2  # friction factor

        # formula 5.82 on page 226 of the book
        V = -5 / 16 * np.pi * gamma / c_f * g * np.sin(theta_radians[0]) / c[0] * dh_dx
        V[H != Hbreaking] = 0  # function valid where waves are breaking

        return V

    # set the acceptable computational error (ratio)
    f_margin = 0.001  # 0.001 = 0.01%

    # set the size of the figure
    fig = Figure((5, 2.5))

    # call the function that builds the backend.
    check_code_function_RANGE(fig, function_name, correct_function, f_margin)

### Wave orbital velocity near bed

In [ ]:
H0, T, d0, slope, angle, rho, r = 1.5, 6, 20, 1 / 30, 10, 1025, 0.06
x_range = np.arange(0, 900, 1)


def W3_plot_u0():
    # define the name of the function that the students will make
    function_name = "W3_u0"

    # define the correct function
    def correct_function(x_range, H0, T, d0, slope, angle):

        # The environmental conditions
        x = x_range  # the horizontal axis
        zbed = -(d0 - slope * x)  # bed elevation [m]
        h = -zbed  # still water depth [m]
        h[h < 0] = 0  # no negative depths

        # given conditions
        gamma = 0.8  # wave breaking ratio

        # The wave characteristics at every location in the cross-section
        L = np.array([wave_length(T, h) for h in h])  # The wavelength
        c = L / T  # The wave celerity
        k = 2 * np.pi / L  # The wave number
        n = 0.5 + (k * h / np.sinh(2 * k * h))
        cg = n * c  # The wave group celerity
        Ksh = np.sqrt(cg[0] / cg)  # The shoaling parameter

        snell_constant = np.sin(np.deg2rad(angle)) / c[0]  # apply snell's law
        theta_radians = np.arcsin(snell_constant * c)
        Kr = (np.cos(theta_radians[0]) / np.cos(theta_radians)) ** 0.5

        H = H0 * Ksh * Kr  # The wave height due to shoaling and refraction
        Hbreaking = gamma * h  # The wave-breaking height
        H[H > Hbreaking] = Hbreaking[H > Hbreaking]  # Adjusting the wave height
        g = 9.81
        E = 1 / 8 * rho * g * H**2  # The wave energy

        # page 199 of the book
        omega = 2 * np.pi / T
        u0 = 0.5 * omega * H / (np.sin(theta_radians) * k * h)

        return u0

    # set the acceptable computational error (ratio)
    f_margin = 0.001  # 0.001 = 0.01%

    # set the size of the figure
    fig = Figure((5, 2.5))

    # call the function that builds the backend.
    check_code_function_RANGE(fig, function_name, correct_function, f_margin)

### Current underneath breaking waves

### Include wave setup

In [ ]:
H0, T, d0, slope, angle, rho = 1.5, 6, 20, 1 / 30, 10, 1025
x_range = np.arange(0, 900, 1)


def W3_plot_wave_setup2():
    # define the name of the function that the students will make
    function_name = "W3_wave_setup2"

    # define the correct function
    def correct_function(x_range, H0, T, d0, slope, angle):
        # The environmental conditions
        x = x_range.copy()  # the horizontal axis
        zbed = -(d0 - slope * x)  # bed elevation [m]
        h = -zbed  # still water depth [m]
        h[h < 0] = 0  # no negative depths
        setup = np.zeros(len(h))  # initial wave-induced setup

        # given conditions
        gamma = 0.8  # wave breaking ratio

        for i in range(3):  # number of iterations considered
            # print(h) # for debugging: print initial water depth each iteration

            # The wave characteristics at every location in the cross-section
            L = np.array([wave_length(T, h) for h in h])  # The wavelength
            c = L / T  # The wave celerity
            k = 2 * np.pi / L  # The wave number
            n = 0.5 + (k * h / np.sinh(2 * k * h))
            cg = n * c  # The wave group celerity
            Ksh = np.sqrt(cg[0] / cg)  # The shoaling parameter

            snell_constant = np.sin(np.deg2rad(angle)) / c[0]  # apply snell's law
            theta_radians = np.arcsin(snell_constant * c)
            Kr = (np.cos(theta_radians[0]) / np.cos(theta_radians)) ** 0.5

            H = H0 * Ksh * Kr  # The wave height due to shoaling and refraction
            Hbreaking = gamma * h  # The wave-breaking height
            H[H > Hbreaking] = Hbreaking[H > Hbreaking]  # Adjusting the wave height
            g = 9.81
            E = 1 / 8 * rho * g * H**2  # The wave energy

            Sxx = (n - 0.5 + n * np.cos(theta_radians) ** 2) * E  # Radiant stresses

            setup = np.zeros(
                Sxx.shape
            )  # here we create a vector for the mean water level
            for i in range(len(setup) - 1):  # key here is that setup[0] = 0
                setup[i + 1] = setup[i] - (Sxx[i + 1] - Sxx[i]) / (1000 * g * h[i])

            setup_shoreline = setup[~np.isnan(setup)][
                -1
            ]  # get setup at the shoreline, which is the last non-Nan value
            setup[np.isnan(setup)] = (
                setup_shoreline  # apply the setup to the water level
            )

            h = -zbed + setup  # the still water depth + setup
            h[np.isnan(h)] = 0
            h[h < 0] = 0
            # plt.plot(h) # for debugging: plot the final water depth each iteration

        return setup

    # set the acceptable computational error (ratio)
    f_margin = 0.01  # 0.001 = 0.01%

    # set the size of the figure
    fig = Figure((5, 2.5))

    # call the function that builds the backend.
    check_code_function_RANGE(fig, function_name, correct_function, f_margin)

Possible student answer